# Convert To Pandas.

This content is a part of Facebook Getting Data's project (First's file is "get_facebook_data.ipynb"). This file cover converter(JSON to Pandas), Reader files(JSON's files) and writer files(Excel's files).

Table of Contents.
* [Import Packages.](#Import-Packages.)
* [Create JSON's File Reader Function.](#Create-JSON's-File-Reader-Function.)
* [Create Excel's File Writer Function.](#Create-Excel's-File-Writer-Function.)
* [Create Convert JSON to Pandas (Comment File) Function.](#Create-Convert-JSON-to-Pandas-(Comment-File%29-Function.)
* [Create Convert JSON to Pandas (Reaction File) Function.](#Create-Convert-JSON-to-Pandas-(Reaction-File%29-Function.)
* [Create Convert JSON to Pandas (Post File) Function.](#Create-Convert-JSON-to-Pandas-(Post-File%29-Function.)


## Import Packages.
Import following packages to get the propose.
    1. Pandas - Pandas is a Helpful datatype. It use for data like table.
    2. OS - In this file, We use to list files.
    3. JSON - The data of facebook is JSON format. We need this to work with them.

In [1]:
import pandas as pd
import os 
import json

## Create JSON's File Reader Function.

In "get_facebook_data.ipynb" file, We write data into JSON's files. This function use to read them.

Input:
    1. filename - Name of file with path.

Output:

    This function will return the data of file in Dict's datatype.

In [6]:
#Function read_jsonfile() , get filename and return data of files (Dict datatype).
def read_jsonfile(filename):
    # Read data in file to variable.
    with open(filename, encoding='utf8') as open_file:
        data_json = json.load(open_file)
        open_file.close()
    
    #Return variable ( Dict datatype ).
    return data_json

## Create Excel's File Writer Function.

This function use to write Pandas's dataframe into EXCEL's format file.

Input:
    1. Data_df - Data in Pandas Dataframe's datatype.
    2. Name - Filename without file extension, It will be file's name and sheet's name.

In [3]:
#Function write_excel(), get data (Pandas's dataframe) and name of filename.
def write_excel(data_df, name):
    #Filename = name + file extension
    filename = name + '.xlsx'
    
    #Write file.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df.to_excel(writer, sheet_name=name)
    
    #Save file.
    writer.save()

## Create Convert JSON to Pandas (Comment File) Function.

This function use to convert JSON to Pandas on Comment's data.

In [4]:
#Function convert_comment_pd(), get comment's data (Dict datatype) and return Pandas's dataframe of the data.
def convert_comment_pd(comment_json):
    #Read json into Pandas's dataframe. Member are member of data's tag of JSON.
    df = pd.io.json.json_normalize(comment_json, 'data', ['post_id','as_of_date'])
    
    #Separate id's, name's tags in from's column to be user_id's and user_name's columns.
    df_from = df['from'].apply(pd.Series)
    df_from.columns = ['user_id','user_name']
    df = pd.concat([df, df_from], axis=1)
    df = df.drop('from', axis=1)
    
    #Order columns.
    df = df.reindex_axis(['as_of_date','post_id','id','created_time','user_id','user_name', 'message', 'like_count'], axis=1)
    
    #Rename columns.
    df.columns = ['as_of_date','post_id','comment_id','created_time','user_id','user_name','message','like_count']
    
    #Return pandas's dataframe.
    return df

## Create Convert JSON to Pandas (Reaction File) Function.

This function use to convert JSON to Pandas on Reaction's data.

In [7]:
#Function convert_reaction_pd(), get reaction's data (Dict datatype) and return Pandas's dataframe of the data.
def convert_reaction_pd(reaction_json):
    #Read json into Pandas's dataframe. Member are member of data's tag of JSON.
    df = pd.io.json.json_normalize(reaction_json, 'data', ['post_id','as_of_date'])
    
    #Order columns.
    df = df.reindex_axis(['as_of_date','post_id','id','name', 'type'], axis=1)

    #Rename columns.
    df.columns = ['as_of_date','post_id','user_id','user_name','type']
    
    #Return pandas's dataframe.
    return df

## Create Convert JSON to Pandas (Post File) Function.

This function use to convert JSON to Pandas on Post's data.

In [1]:
#Function convert_post_pd(), get post's data (Dict datatype) and return Pandas's dataframe of the data.
def convert_post_pd(post_json):
    #Read json into Pandas's dataframe. Member are member of data's tag of JSON.
    df = pd.io.json.json_normalize(post_json, 'data', ['page_id','as_of_date'])

    #Get like_count's tags in likes's column (Under summary's tag) to be like_count's columns.
    df_like = df['likes'].apply(pd.Series)
    df_like.columns = ['data','summary']
    df_summary = df_like['summary'].apply(pd.Series)
    df_summary.columns = ['like_count']
    df = pd.concat([df, df_summary], axis=1)
    df = df.drop('likes', axis=1)

    #Get count's tag in likes's column to be count's columns.
    df_shares = df['shares'].apply(pd.Series, index=['count'])
    df = pd.concat([df, df_shares], axis=1)
    df = df.drop('shares', axis=1)

    #Default NaN to be  Zero for like_count's and count's columns.
    df['like_count'] = df['like_count'].fillna(0)
    df['count'] = df['count'].fillna(0)

    #Order columns.
    df = df.reindex_axis(['as_of_date','created_time','page_id','id', 'name', 'message', 'type', 'like_count', 'count'], axis=1)

    #Rename columns.
    df.columns = ['as_of_date','created_time','page_id','post_id', 'name', 'message', 'type', 'like_count', 'shares_count']
    
    #Return pandas's dataframe.
    return df